In [1]:
import pandas as pd

In [2]:
# Load the HDF5 file using pandas
df = pd.read_hdf('METR-LA.h5', key='df')

print(df.shape)
print(df.head())


(34272, 207)
                        773869     767541     767542     717447     717446  \
2012-03-01 00:00:00  64.375000  67.625000  67.125000  61.500000  66.875000   
2012-03-01 00:05:00  62.666667  68.555556  65.444444  62.444444  64.444444   
2012-03-01 00:10:00  64.000000  63.750000  60.000000  59.000000  66.500000   
2012-03-01 00:15:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2012-03-01 00:20:00   0.000000   0.000000   0.000000   0.000000   0.000000   

                        717445  773062  767620     737529     717816  ...  \
2012-03-01 00:00:00  68.750000  65.125  67.125  59.625000  62.750000  ...   
2012-03-01 00:05:00  68.111111  65.000  65.000  57.444444  63.333333  ...   
2012-03-01 00:10:00  66.250000  64.500  64.250  63.875000  65.375000  ...   
2012-03-01 00:15:00   0.000000   0.000   0.000   0.000000   0.000000  ...   
2012-03-01 00:20:00   0.000000   0.000   0.000   0.000000   0.000000  ...   

                        772167  769372     774204     7

In [3]:
data_np = df.values  # shape (time_steps, num_sensors)

# Normalize, expand dims, etc.
data_min = data_np.min()
data_max = data_np.max()
data_norm = (data_np - data_min) / (data_max - data_min)
data_norm = data_norm[:, :, None]  # add feature dimension

print("Normalized data shape:", data_norm.shape)

Normalized data shape: (34272, 207, 1)


In [4]:
import pickle

with open('adj_METR-LA.pkl', 'rb') as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding='latin1')  # or 'bytes' for Py2 compatibility

print("Adjacency matrix shape:", adj_mx.shape)


Adjacency matrix shape: (207, 207)


In [5]:
import numpy as np

# Parameters
history_len = 12   # input steps (1 hour)
predict_len = 12   # output steps (1 hour)

seq_len = history_len + predict_len

num_samples = data_norm.shape[0]
num_train = int(num_samples * 0.7)
num_val = int(num_samples * 0.1)

train_data = data_norm[:num_train]
val_data = data_norm[num_train - seq_len:]
test_data = data_norm[num_train + num_val - seq_len:]


In [6]:
def generate_sequences(data, history_len=12, predict_len=12):
    inputs = []
    targets = []
    for i in range(len(data) - history_len - predict_len):
        x = data[i : i + history_len]
        y = data[i + history_len : i + history_len + predict_len]
        inputs.append(x)
        targets.append(y)
    return np.array(inputs), np.array(targets)


In [7]:
x_train, y_train = generate_sequences(train_data)
x_val, y_val = generate_sequences(val_data)
x_test, y_test = generate_sequences(test_data)

print(f"Train: {x_train.shape}, {y_train.shape}")
print(f"Val:   {x_val.shape}, {y_val.shape}")
print(f"Test:  {x_test.shape}, {y_test.shape}")


Train: (23966, 12, 207, 1), (23966, 12, 207, 1)
Val:   (10282, 12, 207, 1), (10282, 12, 207, 1)
Test:  (6855, 12, 207, 1), (6855, 12, 207, 1)


In [8]:
import torch
from torch.utils.data import Dataset

class DCRNNDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        # Removed the print statement here
        return self.x[idx], self.y[idx]

In [9]:
from torch.utils.data import DataLoader

batch_size = 64

train_loader = DataLoader(DCRNNDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(DCRNNDataset(x_val, y_val), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(DCRNNDataset(x_test, y_test), batch_size=batch_size, shuffle=False)


Step: Preprocess Adjacency Matrix (Diffusion Supports)


The original Laplacian computation involved dense matrix multiplications using np.diag() and matrix products like D^(-1/2) * A * D^(-1/2), which caused the Jupyter kernel to crash due to high memory usage. This issue typically arises when processing even moderately sized graphs, especially in notebook environments or on GPUs with limited memory.

How it was changed:
To address this, we replaced the matrix multiplication with a more memory-efficient element-wise formulation:

In [11]:
import torch

def compute_scaled_laplacian(adj):
    """
    Compute the scaled symmetric normalized Laplacian.
    This version avoids dense matrix multiplication and is memory-safe.
    Method suggested by OpenAI.
    """
    adj = adj.astype(np.float32)
    n = adj.shape[0]
    d = np.sum(adj, axis=1)
    
    # Avoid division by zero
    d_inv_sqrt = np.zeros_like(d)
    d_inv_sqrt[d > 0] = np.power(d[d > 0], -0.5)

    # Element-wise Laplacian calculation: L = I - D^{-1/2} A D^{-1/2}
    laplacian = np.eye(n, dtype=np.float32) - (d_inv_sqrt[:, None] * adj * d_inv_sqrt[None, :])
    return laplacian

def compute_diffusion_supports(adj_mx):
    """
    Returns [forward_laplacian, reverse_laplacian] as torch tensors.
    """
    forward = compute_scaled_laplacian(adj_mx)
    reverse = compute_scaled_laplacian(adj_mx.T)

    supports = [
        torch.tensor(forward, dtype=torch.float32),
        torch.tensor(reverse, dtype=torch.float32)
    ]
    return supports

In [12]:
supports = compute_diffusion_supports(adj_mx)

# (optional) move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
supports = [s.to(device) for s in supports]

Step 2: Diffusion Convolution Layer

In [13]:
import torch.nn as nn

class DiffusionConv(nn.Module):
    def __init__(self, in_channels, out_channels, diffusion_steps, num_nodes):
        super().__init__()
        self.num_nodes = num_nodes
        self.diffusion_steps = diffusion_steps
        self.in_channels = in_channels
        self.out_channels = out_channels
        # The input to the MLP will be the concatenation of the initial input (x) and the results of the diffusion steps.
        # The output shape of each diffusion step will be (batch_size, num_nodes, in_channels).
        # There are 2 * diffusion_steps diffusion steps (forward and reverse) plus the initial input.
        self.mlp = nn.Linear((diffusion_steps * 2 + 1) * in_channels, out_channels)


    def forward(self, x, supports):
        """
        x: shape (batch_size, num_nodes, in_channels)
        supports: list of precomputed diffusion matrices [forward, reverse], each (num_nodes, num_nodes)
        """
        out = [x]  # 0-hop (identity)
        # print("x shape before einsum:", x.shape) # Debugging line removed

        for support in supports:
            x1 = x
            for k in range(1, self.diffusion_steps + 1):
                # The einsum equation "nm,bmc->bnc" is for multiplying a matrix (support, nm) with a batch of matrices (x1, bmc).
                # support has shape (num_nodes, num_nodes) -> nm
                # x1 has shape (batch_size, num_nodes, in_channels) -> bmc
                # The output should have shape (batch_size, num_nodes, in_channels) -> bnc

                x1 = torch.einsum("nm,bmc->bnc", support, x1)
                 # graph conv

                out.append(x1)
        x_cat = torch.cat(out, dim=-1)  # concat along features
        return self.mlp(x_cat)

In [14]:
class DCRNNCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, diffusion_steps, num_nodes):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_nodes = num_nodes

        self.dc_xz = DiffusionConv(input_dim, hidden_dim, diffusion_steps, num_nodes)
        self.dc_hz = DiffusionConv(hidden_dim, hidden_dim, diffusion_steps, num_nodes)

        self.dc_xr = DiffusionConv(input_dim, hidden_dim, diffusion_steps, num_nodes)
        self.dc_hr = DiffusionConv(hidden_dim, hidden_dim, diffusion_steps, num_nodes)

        self.dc_xh = DiffusionConv(input_dim, hidden_dim, diffusion_steps, num_nodes)
        self.dc_hh = DiffusionConv(hidden_dim, hidden_dim, diffusion_steps, num_nodes)

    def forward(self, x, h, supports):
        z = torch.sigmoid(self.dc_xz(x, supports) + self.dc_hz(h, supports))
        r = torch.sigmoid(self.dc_xr(x, supports) + self.dc_hr(h, supports))
        h_tilde = torch.tanh(self.dc_xh(x, supports) + self.dc_hh(r * h, supports))
        h = (1 - z) * h + z * h_tilde
        return h

In [15]:
import torch
import torch.nn as nn

class DCRNNEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, diffusion_steps, num_nodes):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_nodes = num_nodes
        self.cell = DCRNNCell(input_dim, hidden_dim, diffusion_steps, num_nodes)

    def forward(self, inputs, hidden_state, supports):
        """
        inputs: (batch_size, seq_len, num_nodes, input_dim)
        hidden_state: (batch_size, num_nodes, hidden_dim)
        supports: list of diffusion support matrices
        """
        seq_len = inputs.shape[1]
        h = hidden_state
        for t in range(seq_len):
            x_t = inputs[:, t, :, :]  # (batch, nodes, input_dim)
            h = self.cell(x_t, h, supports)
        return h

In [16]:
batch_size = 64
seq_len = 12
num_nodes = 207
input_dim = 1
hidden_dim = 64
diffusion_steps = 2

encoder = DCRNNEncoder(input_dim, hidden_dim, diffusion_steps, num_nodes)

# Initialize hidden state (zeros)
h0 = torch.zeros(batch_size, num_nodes, hidden_dim)

# Example input batch (random)
x = torch.randn(batch_size, seq_len, num_nodes, input_dim)

# supports is your list of diffusion matrices loaded earlier
output_hidden = encoder(x, h0, supports)

print("Encoder output shape:", output_hidden.shape)  # (batch, nodes, hidden_dim)

Encoder output shape: torch.Size([64, 207, 64])


In [17]:
class DCRNNDecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, diffusion_steps, num_nodes, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_nodes = num_nodes
        self.cell = DCRNNCell(input_dim, hidden_dim, diffusion_steps, num_nodes)
        self.projection = nn.Linear(hidden_dim, output_dim)

    def forward(self, inputs, hidden_state, supports, target_len, targets=None, teacher_forcing_ratio=0.5):
        """
        inputs: (batch_size, 1, num_nodes, input_dim) initial input (e.g., last known step)
        hidden_state: (batch_size, num_nodes, hidden_dim) encoder hidden state
        supports: list of diffusion support matrices
        target_len: int, number of steps to predict
        targets: (batch_size, target_len, num_nodes, output_dim) ground truth for teacher forcing (during training)
        teacher_forcing_ratio: probability to use ground truth as next input during training

        Returns:
          outputs: (batch_size, target_len, num_nodes, output_dim)
        """
        batch_size = inputs.shape[0]
        outputs = []
        input_t = inputs  # first input to decoder (usually last observed step)

        h = hidden_state
        for t in range(target_len):
            # Run one step of DCRNNCell
            # input_t shape: (batch, 1, nodes, features) or (batch, nodes, features) if teacher forcing is off and using previous output
            # Squeeze to remove the time dimension if it's present
            input_to_cell = input_t.squeeze(1) if input_t.ndim == 4 else input_t
            h = self.cell(input_to_cell, h, supports)  # input_to_cell shape: (batch, nodes, input_dim) or (batch, nodes, output_dim)
            output = self.projection(h)  # (batch, nodes, output_dim)
            outputs.append(output.unsqueeze(1))  # add time dimension

            # Decide next input (teacher forcing)
            if self.training and targets is not None and t < target_len -1 and torch.rand(1).item() < teacher_forcing_ratio:
                # Use ground truth next step
                input_t = targets[:, t:t+1, :, :] # shape (batch, 1, nodes, output_dim)
            else:
                # Use prediction as next input
                input_t = output.unsqueeze(1) # shape (batch, 1, nodes, output_dim)


        outputs = torch.cat(outputs, dim=1)
        return outputs

In [18]:
class DCRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, diffusion_steps, num_nodes, output_dim):
        super().__init__()
        self.encoder = DCRNNEncoder(input_dim, hidden_dim, diffusion_steps, num_nodes)
        self.decoder = DCRNNDecoder(output_dim, hidden_dim, diffusion_steps, num_nodes, output_dim) # Decoder input_dim should be output_dim

    def forward(self, source, target_len, supports, targets=None, teacher_forcing_ratio=0.5):
        """
        source: (batch, seq_len, num_nodes, input_dim) - input sequence
        target_len: int - number of time steps to predict
        supports: list of adjacency-based diffusion matrices
        targets: (batch, target_len, num_nodes, output_dim) - ground truth for teacher forcing (during training)
        teacher_forcing_ratio: float [0,1] - probability to use ground truth in decoder during training

        Returns:
          outputs: (batch, target_len, num_nodes, output_dim)
        """
        batch_size, seq_len, num_nodes, input_dim = source.shape

        # Encode input sequence
        h0 = torch.zeros(batch_size, num_nodes, self.encoder.hidden_dim, device=source.device)
        encoder_hidden = self.encoder(source, h0, supports)

        # Prepare initial decoder input: last observed time step
        # Use the last step of the source sequence as the initial input to the decoder.
        # Its shape should be (batch_size, 1, num_nodes, input_dim).
        decoder_input = source[:, -1:, :, :]

        # Decode to predict future steps
        # Pass targets to the decoder if available (during training)
        outputs = self.decoder(decoder_input, encoder_hidden, supports, target_len, targets, teacher_forcing_ratio)
        return outputs

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

# Hyperparameters
input_dim = 1
output_dim = 1
hidden_dim = 64
diffusion_steps = 2
num_nodes = 207
batch_size = 64
epochs = 10
teacher_forcing_ratio = 0.5
learning_rate = 0.001

# Instantiate model
model = DCRNN(input_dim, hidden_dim, diffusion_steps, num_nodes, output_dim)
model.train()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

# Assuming you have a device (e.g., GPU) set up
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
supports = [support.to(device) for support in supports]


for epoch in range(epochs):
    total_loss = 0
    for batch_x, batch_y in train_loader:  # from your DataLoader
        optimizer.zero_grad()

        # Move to device if needed
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        # Forward pass: predict future traffic
        # Pass batch_y to the model during training for teacher forcing
        outputs = model(batch_x, target_len=batch_y.shape[1], supports=supports, targets=batch_y, teacher_forcing_ratio=teacher_forcing_ratio)

        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")